## Assignment 2.2 Sourcing data with an API

In [3]:
## Import and concatenate

In [5]:
import pandas as pd 
import numpy as np
import os 
import requests
import json
from datetime import datetime

In [7]:
%%capture
import warnings
warnings.filterwarnings("ignore")

### As a strategic approach, since the data is too big, every second month of the each quarter's first trip data taken into account.

In [9]:
# Create a list with all files in the folder using a list compehension
import glob

filepaths = glob.glob("Data/*.csv")  # Only select CSV files
filepaths

['Data\\202202-citibike-tripdata_1.csv',
 'Data\\202205-citibike-tripdata_1.csv',
 'Data\\202208-citibike-tripdata_1.csv',
 'Data\\202211-citibike-tripdata_1.csv']

### We used a generator to list comprehensions, since we need to iterate through the files just once, and generators are capable of handling one file at a time

In [12]:
df = pd.concat((pd.read_csv(f) for f in filepaths), ignore_index=True)
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,70F3BA79664016C1,classic_bike,2022-02-28 09:22:49.821,2022-02-28 09:28:55.174,9 Ave & W 18 St,6190.08,Greenwich Ave & Charles St,5914.08,40.743174,-74.003664,40.735238,-74.000271,member
1,8FA3412A958122D0,electric_bike,2022-02-14 10:47:30.462,2022-02-14 10:56:04.433,W 120 St & Claremont Ave,7745.07,Frederick Douglass Blvd & W 117 St,7688.12,40.810949,-73.963400,40.805159,-73.954692,member
2,0E2BA02AEAD03EB8,electric_bike,2022-02-18 22:26:55.796,2022-02-18 22:44:31.472,E 4 St & 2 Ave,5593.04,Greenwich Ave & Charles St,5914.08,40.726281,-73.989780,40.735238,-74.000271,member
3,B626F8E1D7ABABBA,classic_bike,2022-02-09 18:38:18.621,2022-02-09 18:47:11.274,White St & Johnson Ave,5099.03,Irving Ave & Harman St,4856.05,40.707174,-73.935810,40.701080,-73.917900,member
4,605E7CC493D08F3B,electric_bike,2022-02-06 14:52:48.564,2022-02-06 14:55:56.218,S Portland Ave & Hanson Pl,4354.05,Nevins St & Schermerhorn St,4437.09,40.685396,-73.974315,40.687372,-73.981761,member


### We identify the available csv files with the data path. Then we went over the loop to read for each csv file row by row and concatenated it.

In [15]:
df.shape

(4000000, 13)

In [17]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,70F3BA79664016C1,classic_bike,2022-02-28 09:22:49.821,2022-02-28 09:28:55.174,9 Ave & W 18 St,6190.08,Greenwich Ave & Charles St,5914.08,40.743174,-74.003664,40.735238,-74.000271,member
1,8FA3412A958122D0,electric_bike,2022-02-14 10:47:30.462,2022-02-14 10:56:04.433,W 120 St & Claremont Ave,7745.07,Frederick Douglass Blvd & W 117 St,7688.12,40.810949,-73.963400,40.805159,-73.954692,member
2,0E2BA02AEAD03EB8,electric_bike,2022-02-18 22:26:55.796,2022-02-18 22:44:31.472,E 4 St & 2 Ave,5593.04,Greenwich Ave & Charles St,5914.08,40.726281,-73.989780,40.735238,-74.000271,member
3,B626F8E1D7ABABBA,classic_bike,2022-02-09 18:38:18.621,2022-02-09 18:47:11.274,White St & Johnson Ave,5099.03,Irving Ave & Harman St,4856.05,40.707174,-73.935810,40.701080,-73.917900,member
4,605E7CC493D08F3B,electric_bike,2022-02-06 14:52:48.564,2022-02-06 14:55:56.218,S Portland Ave & Hanson Pl,4354.05,Nevins St & Schermerhorn St,4437.09,40.685396,-73.974315,40.687372,-73.981761,member


In [19]:
df.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
3999995,C69A2CD7EC205647,electric_bike,2022-11-04 19:27:39.062,2022-11-04 19:46:03.141,Carlton Ave & Park Ave,4732.04,Kent Ave & N 7 St,5489.03,40.695807,-73.973556,40.720368,-73.961651,member
3999996,746CF0556074D6C2,electric_bike,2022-11-22 15:54:17.752,2022-11-22 16:00:07.050,W 30 St & 8 Ave,6373.07,Lexington Ave & E 26 St,6089.08,40.749653,-73.995208,40.741459,-73.983293,member
3999997,1754E82C4F5AE0F5,electric_bike,2022-11-04 14:29:38.255,2022-11-04 14:44:22.117,Park Ave & E 124 St,7682.01,W 113 St & Broadway,7713.01,40.804604,-73.939631,40.805973,-73.964928,member
3999998,3939C8FA9E69C8DD,electric_bike,2022-11-23 11:12:48.626,2022-11-23 11:22:18.603,Amsterdam Ave & W 136 St,7920.05,Gerard Ave & E 146 St,7868.05,40.819746,-73.951819,40.817755,-73.929650,member
3999999,4D7B1544F05D6461,electric_bike,2022-11-30 17:44:54.489,2022-11-30 18:00:45.990,8 Ave & W 49 St,6747.07,E 47 St & Park Ave,6584.12,40.761823,-73.986487,40.755103,-73.974987,member


### Heads and Tails match, so they merged perfectly

In [22]:
df = pd.concat(
    (pd.read_csv(f, dtype={"start_station_name": str, "end_station_name": str}, low_memory=False) for f in filepaths),
    ignore_index=True
)


In [23]:
print(df.dtypes)  # Should show 'object' for start_station_name and end_station_name
print(df[['start_station_name', 'end_station_name']].head())  # Preview data


ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object
           start_station_name                    end_station_name
0             9 Ave & W 18 St          Greenwich Ave & Charles St
1    W 120 St & Claremont Ave  Frederick Douglass Blvd & W 117 St
2              E 4 St & 2 Ave          Greenwich Ave & Charles St
3      White St & Johnson Ave              Irving Ave & Harman St
4  S Portland Ave & Hanson Pl         Nevins St & Schermerhorn St


In [24]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,70F3BA79664016C1,classic_bike,2022-02-28 09:22:49.821,2022-02-28 09:28:55.174,9 Ave & W 18 St,6190.08,Greenwich Ave & Charles St,5914.08,40.743174,-74.003664,40.735238,-74.000271,member
1,8FA3412A958122D0,electric_bike,2022-02-14 10:47:30.462,2022-02-14 10:56:04.433,W 120 St & Claremont Ave,7745.07,Frederick Douglass Blvd & W 117 St,7688.12,40.810949,-73.963400,40.805159,-73.954692,member
2,0E2BA02AEAD03EB8,electric_bike,2022-02-18 22:26:55.796,2022-02-18 22:44:31.472,E 4 St & 2 Ave,5593.04,Greenwich Ave & Charles St,5914.08,40.726281,-73.989780,40.735238,-74.000271,member
3,B626F8E1D7ABABBA,classic_bike,2022-02-09 18:38:18.621,2022-02-09 18:47:11.274,White St & Johnson Ave,5099.03,Irving Ave & Harman St,4856.05,40.707174,-73.935810,40.701080,-73.917900,member
4,605E7CC493D08F3B,electric_bike,2022-02-06 14:52:48.564,2022-02-06 14:55:56.218,S Portland Ave & Hanson Pl,4354.05,Nevins St & Schermerhorn St,4437.09,40.685396,-73.974315,40.687372,-73.981761,member


In [28]:
df.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
3999995,C69A2CD7EC205647,electric_bike,2022-11-04 19:27:39.062,2022-11-04 19:46:03.141,Carlton Ave & Park Ave,4732.04,Kent Ave & N 7 St,5489.03,40.695807,-73.973556,40.720368,-73.961651,member
3999996,746CF0556074D6C2,electric_bike,2022-11-22 15:54:17.752,2022-11-22 16:00:07.050,W 30 St & 8 Ave,6373.07,Lexington Ave & E 26 St,6089.08,40.749653,-73.995208,40.741459,-73.983293,member
3999997,1754E82C4F5AE0F5,electric_bike,2022-11-04 14:29:38.255,2022-11-04 14:44:22.117,Park Ave & E 124 St,7682.01,W 113 St & Broadway,7713.01,40.804604,-73.939631,40.805973,-73.964928,member
3999998,3939C8FA9E69C8DD,electric_bike,2022-11-23 11:12:48.626,2022-11-23 11:22:18.603,Amsterdam Ave & W 136 St,7920.05,Gerard Ave & E 146 St,7868.05,40.819746,-73.951819,40.817755,-73.929650,member
3999999,4D7B1544F05D6461,electric_bike,2022-11-30 17:44:54.489,2022-11-30 18:00:45.990,8 Ave & W 49 St,6747.07,E 47 St & Park Ave,6584.12,40.761823,-73.986487,40.755103,-73.974987,member


## ## Get weather data using NOAA's API

In [31]:
# Define your NOAA token - type yours in here! 

Token = 'GCWhOUMGHMLILXunaHDbUZACXUbnFrOa' 
stationid="GHCND:USW00014732"

In [33]:
# Get the API 

r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31', headers={'token':Token})

In [35]:
# Load the api response as a json

d = json.loads(r.text)

In [37]:
# Secure all items in the response that correspond to TAVG

avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']

In [39]:
# Get only the date field from all average temperature readings

dates_temp = [item['date'] for item in avg_temps]

In [41]:
# Get the temperature from all average temperature readings

temps = [item['value'] for item in avg_temps]

In [43]:
temps[10:20]

[-54, -19, 40, 48, -67, -80, 39, 18, 32, 51]

In [45]:
# Put the results in a dataframe

df_temp = pd.DataFrame()

In [47]:
# Get only date and cast it to date time; convert temperature from tenths of Celsius to normal Celsius

df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_temp['avgTemp'] = [float(v)/10.0 for v in temps]

In [49]:
df_temp.tail()

,date,avgTemp
360,2022-12-27,-0.7
361,2022-12-28,3.4
362,2022-12-29,6.4
363,2022-12-30,9.3
364,2022-12-31,8.2


In [51]:
df_temp.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


In [53]:
df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [55]:
df['started_at'] = pd.to_datetime(df['started_at'], dayfirst = True)

In [57]:
df['date'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d').dt.date

In [59]:
df['date'] = pd.to_datetime(df['date'])

In [61]:
df_temp.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


In [63]:
%%time
df_merged = df.merge(df_temp, how = 'left', on = 'date', indicator = True)

CPU times: total: 3.62 s
Wall time: 4.03 s


In [65]:
df_merged.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,_merge
0,70F3BA79664016C1,classic_bike,2022-02-28 09:22:49.821,2022-02-28 09:28:55.174,9 Ave & W 18 St,6190.08,Greenwich Ave & Charles St,5914.08,40.743174,-74.003664,40.735238,-74.000271,member,2022-02-28,0.8,both
1,8FA3412A958122D0,electric_bike,2022-02-14 10:47:30.462,2022-02-14 10:56:04.433,W 120 St & Claremont Ave,7745.07,Frederick Douglass Blvd & W 117 St,7688.12,40.810949,-73.963400,40.805159,-73.954692,member,2022-02-14,-4.3,both
2,0E2BA02AEAD03EB8,electric_bike,2022-02-18 22:26:55.796,2022-02-18 22:44:31.472,E 4 St & 2 Ave,5593.04,Greenwich Ave & Charles St,5914.08,40.726281,-73.989780,40.735238,-74.000271,member,2022-02-18,9.9,both
3,B626F8E1D7ABABBA,classic_bike,2022-02-09 18:38:18.621,2022-02-09 18:47:11.274,White St & Johnson Ave,5099.03,Irving Ave & Harman St,4856.05,40.707174,-73.935810,40.701080,-73.917900,member,2022-02-09,2.8,both
4,605E7CC493D08F3B,electric_bike,2022-02-06 14:52:48.564,2022-02-06 14:55:56.218,S Portland Ave & Hanson Pl,4354.05,Nevins St & Schermerhorn St,4437.09,40.685396,-73.974315,40.687372,-73.981761,member,2022-02-06,-4.8,both


### Average temperature and start time and date are merged.

In [68]:
df_merged['_merge'].value_counts(dropna = False)

both          3999995
left_only           5
right_only          0
Name: _merge, dtype: int64

In [70]:
df_merged.to_csv('Data/output/newyork_data.csv', index = False)

In [72]:
df.shape

(4000000, 14)

In [74]:
df_temp.to_csv('Data/output/nytemperature.csv', index = False)